In [1]:
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/MyDrive/mPDE-Net/')
sys.path.append('/content/drive/MyDrive/mPDE-Net/aTEAM/')
!pip install timeout_decorator
!pip install -U PyYAML

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#mPDE-Net

In [2]:

import sys,time
import numpy as np
import torch
import timeout_decorator
from aTEAM.optim import NumpyFunctionInterface #NFI_ex21d1
from scipy.optimize.lbfgsb import fmin_l_bfgs_b as lbfgsb
from scipy.optimize.slsqp import fmin_slsqp as slsqp
from scipy.optimize import fmin_bfgs as bfgs
import conf



import setenv21d1 as setenv

import initparameters_ex21d1 as initparameters

kw = {
          '--name':'Murrayll28',
#         '--dtype':'double',
#         '--device':'cuda:0',
          '--constraint':'2',
#         # computing region
          '--eps':2*5, #2*np.pi,
         '--dt':1e-2/4,
#         '--cell_num':1,
          '--blocks':'0-6,9,12,15',
#         # super parameters of network
          '--kernel_size':5,
#         '--max_order':2,
          '--dx':2*5/64,#2*np.pi/32,
          '--hidden_layers':8,
#         '--scheme':'upwind',
#         # data generator
          '--dataname':'Murrayll28',
#         '--viscosity':0.05,
          '--zoom':1,
          '--max_dt':1/1600,
          '--batch_size':12,
#         '--data_timescheme':'rk2',
#         '--channel_names':'u,v',
         '--freq':13,
          '--data_start_time':0,
#         # data transform
          '--start_noise':0.05,
          '--end_noise':0.05,#0.0,
        #   '--stablize':4.21e-08,#noise 0
        #  '--sparsity':0.000799,
        #  '--momentsparsity':0.000546,
         '--stablize':5.46e-05,#005
         '--sparsity':4.93e-05,
         '--momentsparsity':3.28e-05,
        #  '--npseed':-1,
        #  '--torchseed':-1,
          '--maxiter':2000,
#         '--recordfile':'None',
          '--recordcycle':200,
#         '--savecycle':-1,
#         '--start_from':-1,
          '--mesh_size': [64, 64],
          "--boundary_condition": "Neumann",
          "--lower_bound": -5,
          "--upper_bound": 5,
          "--dim":"2d",



          }
options = conf.setoptions(argv=sys.argv[1:],kw=kw,configfile=None)

if torch.cuda.is_available():
    options['--device'] = 'cuda'
else:
    options['--device'] = 'cpu'

globalnames, callback, model, data_model, sampling, addnoise,addnoiset = setenv.setenv(options)
globals().update(globalnames)


torch.cuda.manual_seed_all(torchseed)
torch.manual_seed(torchseed)
np.random.seed(npseed)

# initialization of parameters
if start_from<0:

    initparameters.initkernels(model, scheme=scheme)

    initparameters.initexpr(model, viscosity=viscosity, pattern='random')

else: # load checkpoint of layer-$start_from
    callback.load(start_from, iternum='final')
ftestloss=[]
tu_obs,tu_true,tu = \
        setenv.data(model,data_model,globalnames,sampling,addnoise,blocks[-1],data_start_time)
tutest_obs,tutest_true,tutest = \
    setenv.cvtestdata(model,data_model,globalnames,sampling,addnoiset,blocks[-1],data_start_time)

#%% train
for block in blocks:
    trainlossf=[]
    testlossf=[]
    if block<=start_from:
        continue
    r = np.random.randn()+torch.randn(1,dtype=torch.float64,device=device).item()
    with callback.open() as output:
        print('device: ', device, file=output)
        print('generate a random number to check random seed: ', r, file=output)
    if block == 0:
        callback.stage = 'warmup'
        isfrozen = (False if constraint == 'free' else True)
        #isfrozen = False
    else:
        callback.stage = 'block-'+str(block)
        isfrozen = False
        if constraint == 'frozen':
            isfrozen = True
    stepnum = block if block>=1 else 1
    layerweight = [1,]*stepnum


    u_obs,u_true,u = \
        tu_obs[:stepnum+1],tu_true[:stepnum+1],tu[:stepnum+1]
    utest_obs,utest_true,utest = \
        tutest_obs[:stepnum+1],tutest_true[:stepnum+1],tutest[:stepnum+1]
    print("utest_obs", utest_obs[0].shape)
    print("u_obs shape: batchsize x channelNum x xgridsize x ygridsize")
    print(u_obs[0].shape)
    print("u_obs.abs().max()")
    print(u_obs[0][:,0,:].abs().max())
    print("v_obs.abs().max()")
    print(u_obs[0][:,1,:].abs().max())
    print("u_obs.abs().min()")
    print(u_obs[0][:,0,:].min())
    print("v_obs.abs().min()")
    print(u_obs[0][:,1,:].min())
    print("u_lib variance")
    print(initparameters.trainvar(model.UInputs(u_obs[0])))

    print("u_obs variance")
    print(initparameters.trainvar(u_obs[0]))

    # set NumpyFunctionInterface
    def forward():
        stableloss,dataloss,sparseloss,momentloss, testloss = \
                setenv.loss(model, u_obs, utest_obs, globalnames, block, layerweight)
        layersparsity=0
        if block == 0:
            # for stage='warmup', no regularization term used
            loss = dataloss
        else:
            loss = stablize*stableloss+dataloss+stepnum*sparsity*sparseloss+stepnum*momentsparsity*momentloss#+stepnum*layersparsity*lastlayerloss
        if constraint == 'frozen':
            momentloss = 0

            loss = stablize*stableloss+dataloss+stepnum*sparsity*sparseloss+stepnum*momentsparsity*momentloss#+stepnum*layersparsity*lastlayerloss

        if torch.isnan(loss):
            loss = (torch.ones(1,requires_grad=True)/torch.zeros(1)).to(loss)

        tloss=testloss



        testlossf.append(tloss)

        return loss

    nfi = NumpyFunctionInterface([
        dict(params=model.diff_params(), isfrozen=isfrozen,
            x_proj=model.diff_x_proj, grad_proj=model.diff_grad_proj),
        dict(params=model.expr_params(),
            isfrozen=False)
        ], forward=forward, always_refresh=False,block=block, tol=1)
    callback.nfi = nfi

    def callbackhook(_callback, *args):
        # global model,block,u0_obs,T,stable_loss,data_loss,sparse_loss
        stableloss,dataloss,sparseloss,momentloss, testloss = \
                setenv.loss(model, u_obs, utest_obs, globalnames, block, layerweight)
        stableloss,dataloss,sparseloss,momentloss = \
                stableloss.item(),dataloss.item(),sparseloss.item(),momentloss.item()
        with _callback.open() as output:
            print("stableloss: {:.2e}".format(stableloss), "  dataloss: {:.2e}".format(dataloss),
                    "  sparseloss: {:.2e}".format(sparseloss), "momentloss: {:.2e}".format(momentloss),
                    file=output)
        return None
    callbackhookhandle = callback.register_hook(callbackhook)
    if block == 0:
        callback.save(nfi.flat_param, 'start')
    try:

        # optimize
        xopt = bfgs(nfi.f,nfi.flat_param,nfi.fprime,gtol=2e-16,maxiter=maxiter, callback=callback)
        # xopt,f,d = lbfgsb(nfi.f, nfi.flat_param, nfi.fprime, m=maxiter, callback=callback, factr=1e7, pgtol=1e-8,maxiter=maxiter,iprint=0)
        np.set_printoptions(precision=2, linewidth=90)
        print("convolution moment and kernels")
        for k in range(max_order+1):
            for j in range(k+1):
                print((model.__getattr__('fd'+str(j)+str(k-j)).moment).data.cpu().numpy())
                print((model.__getattr__('fd'+str(j)+str(k-j)).kernel).data.cpu().numpy())
        for p in model.expr_params():
            print("SymNet parameters")
            print(p.data.cpu().numpy())
    except RuntimeError as Argument:
        with callback.open() as output:
            print(Argument, file=output) # if overflow then just print and continue
    finally:
        # save parameters
        nfi.flat_param = xopt
        callback.save(xopt, 'final')
        with callback.open() as output:
            print('finally, finish this stage', file=output)
        callback.record(xopt, callback.ITERNUM)
        callbackhookhandle.remove()
        @timeout_decorator.timeout(800)
        def printcoeffs():
            with callback.open() as output:
                print('current expression:', file=output)
                for poly in model.polys:
                    tsym,csym = poly.coeffs()
                    print(tsym[:20], file=output)
                    print(csym[:20], file=output)
        try:
            printcoeffs()
        except timeout_decorator.TimeoutError:
            with callback.open() as output:
                print('Time out', file=output)

    ftestloss.append(testlossf)


with callback.open() as output:
    print("u_obs.abs().max()", file=output)
    print(u_obs[0][:,0,:].abs().max(), file=output)
    print("v_obs.abs().max()", file=output)
    print(u_obs[0][:,1,:].abs().max(), file=output)
with torch.no_grad():
    with callback.open() as output:
        print("model(u_obs[0],T=20*dt).abs().max()", file=output)
        print(model(u_obs[0], T=20*dt).abs().max(), file=output)
        print("model(u_obs[0],T=30*dt).abs().max()", file=output)
        print(model(u_obs[0], T=30*dt).abs().max(), file=output)

<ipython-input-2-fd716b693a75>:6: DeprecationWarning: Please use `fmin_l_bfgs_b` from the `scipy.optimize` namespace, the `scipy.optimize.lbfgsb` namespace is deprecated.
  from scipy.optimize.lbfgsb import fmin_l_bfgs_b as lbfgsb
<ipython-input-2-fd716b693a75>:7: DeprecationWarning: Please use `fmin_slsqp` from the `scipy.optimize` namespace, the `scipy.optimize.slsqp` namespace is deprecated.
  from scipy.optimize.slsqp import fmin_slsqp as slsqp


Brusselator
initgenb
utest_obs torch.Size([3, 2, 64, 64])
u_obs shape: batchsize x channelNum x xgridsize x ygridsize
torch.Size([12, 2, 64, 64])
u_obs.abs().max()
tensor(2.0666, dtype=torch.float64)
v_obs.abs().max()
tensor(5.3383, dtype=torch.float64)
u_obs.abs().min()
tensor(-1.2667, dtype=torch.float64)
v_obs.abs().min()
tensor(1.9809, dtype=torch.float64)
u_lib variance
tensor([ 0.0423,  0.8910,  0.7786, 17.4955,  6.6314, 15.1613,  0.0508,  1.0199,
         1.0419, 19.8258,  9.2510, 20.4589], dtype=torch.float64,
       grad_fn=<MeanBackward1>)
u_obs variance
tensor([0.0423, 0.0508], dtype=torch.float64)


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 31.212386
         Iterations: 95
         Function evaluations: 165
         Gradient evaluations: 153
convolution moment and kernels
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[ 0.    1.    0.   -0.33 -0.25]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.11e-16  2.22e-16 -1.50e+00  2.00e+00 -5.00e-01]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]]
[[ 0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [-0.33  0.    0.    0.    0.  ]
 [-0.25  0.    0.    0.    0.  ]]
[[ 0.00e+00  0.00e+00  1.11e-16  0.00e+00  0.00e+00]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


...

         Current function value: 213.941020
         Iterations: 54
         Function evaluations: 127
         Gradient evaluations: 115
convolution moment and kernels
[[ 1.    0.    0.01 -0.01  0.01]
 [ 0.    0.02  0.03 -0.05  0.07]
 [ 0.02 -0.16 -0.61 -0.33 -0.61]
 [ 0.03 -0.07  0.06 -0.19  0.11]
 [ 0.05 -0.11 -0.3  -0.21 -0.2 ]]
[[-0.14  0.38 -0.47  0.48 -0.22]
 [-0.08  0.47 -0.87  0.19  0.13]
 [ 0.52 -1.88  3.82 -1.33  0.13]
 [-0.33  1.06 -1.47  0.37  0.16]
 [ 0.04 -0.07  0.01  0.3  -0.21]]
[[ 0.00e+00  1.00e+00  0.00e+00 -6.11e-05  4.43e-02]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]]
[[ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.13 -0.84  0.27  0.49 -0.04]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.00e+00  0

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 277.446336
         Iterations: 40
         Function evaluations: 113
         Gradient evaluations: 104
convolution moment and kernels
[[ 1.    0.    0.   -0.   -0.01]
 [ 0.    0.02  0.04 -0.04  0.05]
 [-0.01 -0.18 -0.56 -0.31 -0.55]
 [ 0.02 -0.07  0.06 -0.19  0.08]
 [ 0.01 -0.11 -0.29 -0.21 -0.18]]
[[-0.11  0.28 -0.36  0.38 -0.19]
 [-0.11  0.59 -0.91  0.25  0.13]
 [ 0.51 -1.84  3.57 -1.21  0.07]
 [-0.34  1.07 -1.31  0.31  0.19]
 [ 0.04 -0.06 -0.05  0.3  -0.21]]
[[ 0.00e+00  1.00e+00  0.00e+00 -5.10e-05  4.26e-02]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]]
[[ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.13 -0.84  0.26  0.5  -0.04]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.00e+00  0

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 338.831965
         Iterations: 50
         Function evaluations: 108
         Gradient evaluations: 97
convolution moment and kernels
[[ 1.    0.    0.01  0.    0.  ]
 [ 0.    0.    0.02 -0.05  0.04]
 [ 0.   -0.14 -0.43 -0.24 -0.42]
 [ 0.02 -0.06  0.02 -0.14  0.05]
 [ 0.02 -0.09 -0.22 -0.17 -0.12]]
[[-0.05  0.12 -0.15  0.22 -0.13]
 [-0.18  0.77 -1.11  0.43  0.04]
 [ 0.51 -1.82  3.54 -1.27  0.15]
 [-0.31  1.03 -1.34  0.44  0.09]
 [ 0.04 -0.1   0.05  0.18 -0.15]]
[[ 0.00e+00  1.00e+00  0.00e+00 -2.79e-05  3.69e-02]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]]
[[ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.12 -0.81  0.22  0.52 -0.05]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.00e+00  0.

#Retrieve Coefficients and terms from Trained mFrac-PDE-Net Model


In [4]:
from sympy.parsing.sympy_parser import parse_expr
from sympy import symbols, lambdify
def givevals(dic, setexp):
  u00="u00"
  v00="v00"
  lnu="lnu"
  I="I"

  newlist={}
  for expr in list(setexp.keys()):
    #print(expr)
    dim=64*64*15
    if expr == 1:
        newlist[expr]=np.ones((dim,1))
    elif u00 in str(expr) and v00 in str(expr):
      #print(expr)
      f = lambdify({u00,v00}, expr)
      finalf=f(dic[u00][:,0],dic[v00][:,0])
      newlist[expr]=finalf.reshape((dim,1))
    elif u00 in str(expr) and v00 not in str(expr):
      #print(expr)
      f = lambdify(u00, expr)
      finalf=f(dic[u00][:,0])
      newlist[expr]=finalf.reshape((dim,1))
    elif u00 not in str(expr) and v00 in str(expr):
      #print(expr)
      f = lambdify(v00, expr)
      finalf=f(dic[v00][:,0])
      newlist[expr]=finalf.reshape((dim,1))
    elif "exp"  in str(expr) and "I" not in str(expr):
      f = lambdify(lnu, expr)
      finalf=f(dic[lnu][:,0])
      newlist[expr]=finalf.reshape((dim,1))
    elif "exp"  in str(expr) and "I" in str(expr):
      f = lambdify({lnu,I}, expr)
      finalf=f(channel_dic[lnu][:,0],channel_dic[I][:,0])
      newlist[expr]=finalf.reshape((dim,1))

    else:
      for i in list(dic.keys()):
        if str(i) == str(expr):
          newlist[expr]=dic[i]
  return newlist

In [7]:
#no parsity
termunosp={}
termvnosp={}
termunospc={}
termvnospc={}

#l2 norm
termul2={}
termvl2={}
termul2c={}
termvl2c={}

#sparsity
termusp={}
termvsp={}
termuspc={}
termvspc={}

terms0 = []
terms1 = []

options = {}
options['--name'] = 'Murrayll28'
configfile="checkpoint"+"/"+options['--name']+'/options.yaml'
options = conf.setoptions(argv=None,kw=None,configfile=configfile,isload=True)
options['--device'] = 'cpu'
block = 15

globalnames, callback, model, data_model, sampling, addnoise, addnoiset = setenv.setenv(options)

callback.load(int(block))

tsym,csym = model.poly0.coeffs(calprec=6)
terms0 = tsym[:40]
coeffs0 = np.zeros((len(terms0)))
coeffs0[:min(len(tsym),40)] = csym[:min(len(tsym),40)]
tsym,csym = model.poly1.coeffs(calprec=6)
terms1 = tsym[:40]
coeffs1 = np.zeros((len(terms1)))
coeffs1[:min(len(tsym),40)] = csym[:min(len(tsym),40)]
torch.save(dict(terms0=terms0,terms1=terms1,coeffs0=coeffs0,coeffs1=coeffs1),'-block-000-'+str(block))
lib=torch.load("-block-000-15")


l1=lib['terms0']
l2=lib['terms1']
coe1=lib['coeffs0']
coe2=lib['coeffs1']

In [11]:
simulation=1
sim=0
termuc={}
termvc={}

for order, ele in enumerate(l1):
  if "/" not in str(ele):
    termuc[ele]=1

for order, ele in enumerate(l2):
  if "/" not in str(ele):
    termvc[ele]=1

if len(termunosp)==0:
  termunosp={i:[0]*simulation for i in l1}
  termvnosp={i:[0]*simulation for i in l2}
  for order, ele in enumerate(l1):
    termunosp[ele][sim]=round(coe1[order],5)
  for order, ele in enumerate(l2):
    termvnosp[ele][sim]=round(coe2[order],5)
  termunospc.update(termuc)
  termvnospc.update(termvc)

else:
  for order, ele in enumerate(l1):
    if ele in list(termunosp.keys()):
      termunosp[ele][sim]=round(coe1[order],5)
    else:
      termunosp[ele]=[0]*simulation
      termunosp[ele][sim]=round(coe1[order],5)

  for order, ele in enumerate(l2):
    if ele in list(termvnosp.keys()):
      termvnosp[ele][sim]=round(coe2[order],5)
    else:
      termvnosp[ele]=[0]*simulation
      termvnosp[ele][sim]=round(coe2[order],5)
  for order, item in enumerate(list(termuc)):
    if item in list(termunospc.keys()):
      termunospc[item]+=termuc[item]

    else:
      termunospc[item]=termuc[item]
  for order, item in enumerate(termvc):
    if item in list(termvnospc.keys()):
      termvnospc[item]+=termvc[item]

    else:
      termvnospc[item]=termvc[item]

T = 1e-2
_,_,u = setenv.data(model,data_model,globalnames,sampling,addnoise,block=1,data_start_time=0)
init = u[0]
x0 = init



x1=x0
xtemp=np.zeros((15,12,64,64))
utemp=np.zeros((15,2,64,64))

for t in range(15):
    print('t={:.1e}'.format(t*T))

    startt = time.time()
    with torch.no_grad():
        x1 = data_model.predict(x1, T=T)
        _,x41 = addnoise(x0,x1)
        xtmp4, ut = model.predict_val(x41,T)
        xtmp4=xtmp4[:1,...,...]
        ut=ut[:1,...,...]
        xtemp[t,:,:,:]=xtmp4.data.cpu().numpy()
        utemp[t,:,:,:]=ut.data.cpu().numpy()


        #print("x1", x1)
    stopt = time.time()


allchannels = []
channel_names=model.channel_names
max_order=2

for c in channel_names:
      for k in range(max_order+1):
          for j in range(k+1):
              allchannels.append(c+str(j)+str(k-j))


xnew=xtemp

  #get matrix of X
channel_dic={}
for order, ele in enumerate(xnew[0]):
    channel_dic[allchannels[order]]=xnew[:,order,:,:].T.reshape(64*64*15,1)

rhsu={i:0 for i in l1}
rhsv={i:0 for i in l2}

rhu1=givevals(channel_dic, rhsu)
rhv1=givevals(channel_dic, rhsv)

u20=symbols("u20")
u02=symbols("u02")
v20=symbols("v20")
v02=symbols("v02")
nt=u20+u02
rhu11=rhu1.copy()

indu20=l1.index(u20)
indu02=l1.index(u02)
rhu11[nt]=rhu11.pop(u20)
rhu11[nt]=rhu11[nt]+rhu11[u02]
l1[indu20]=l1[indu20]+l1[indu02]
l1.remove(l1[indu02])

nt=v20+v02
rhv11=rhv1

indv20=l2.index(v20)
indv02=l2.index(v02)
rhv11[nt]=rhv11.pop(v20)
rhv11[nt]=rhv11[nt]+rhv11[v02]
rhv11.pop(v02)
l2[indv20]=l2[indv20]+l2[indv02]
l2.remove(l2[indv02])

dataMatrixu = np.array([rhu11[i] for i in l1])
dataMatrixv = np.array([rhv11[i] for i in l2])

linear_matu=dataMatrixu.T.reshape((64*64*15,len(l1)))
linear_matv=dataMatrixv.T.reshape((64*64*15,len(l2)))

coefu=coeffs0
coefv=coeffs1
coefu=np.delete(coefu,indu20)
coefv=np.delete(coefv,indv20)
coefu=np.reshape(coefu, (-1,1))
coefv=np.reshape(coefv, (-1,1))
# linear_matu1=np.round(linear_matu,4)
# linear_matv1=np.round(linear_matv,4)

uut=linear_matu@coefu
uvt=linear_matv@coefv

initgenb
t=0.0e+00
t=1.0e-02
t=2.0e-02
t=3.0e-02
t=4.0e-02
t=5.0e-02
t=6.0e-02
t=7.0e-02
t=8.0e-02
t=9.0e-02
t=1.0e-01
t=1.1e-01
t=1.2e-01
t=1.3e-01
t=1.4e-01


<ipython-input-4-ea021df2e01b>:17: SymPyDeprecationWarning: 

Passing the function arguments to lambdify() as a set is deprecated. This
leads to unpredictable results since sets are unordered. Instead, use a list
or tuple for the function arguments.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-lambdify-arguments-set
for details.

This has been deprecated since SymPy version 1.6.3. It
will be removed in a future version of SymPy.

  f = lambdify({u00,v00}, expr)


In [12]:
termunosp

{1: [1.01212],
 u00: [-0.98806],
 u00*v00/(1.0*u00**2 + 0.25488950789362*u00 + 0.257304836935724): [-0.50826],
 u20: [0.29521],
 u02: [0.29521],
 v00: [-0.0022],
 v00/(1.0*u00**2 + 0.25488950789362*u00 + 0.257304836935724): [-0.00126]}

In [13]:
termvnosp

{u00*v00/(1.0*u00**2 + 0.252608602212288*u00 + 0.266239556147262): [-0.52235],
 1: [0.40524],
 v20: [0.39134],
 v02: [0.39134],
 v00: [-0.19456],
 u00: [0.01553],
 v00/(1.0*u00**2 + 0.252608602212288*u00 + 0.266239556147262): [-0.00256]}

#$L^2$ norm slection criterion

In [14]:
l2normu=np.sum(np.abs(linear_matu.T)**2,axis=-1)**(1./2)
l2normu1=l2normu.reshape((len(l2normu),1))
scalu=l2normu1*coefu


indtermusp=[]
for order,ele in enumerate(scalu):
  if abs(ele)>max(abs(scalu))/50:

    indtermusp.append(order)

luspy=[]
for i in indtermusp:
  luspy.append(l1[i])


#v
l2normv=np.sum(np.abs(linear_matv.T)**2,axis=-1)**(1./2)
l2normv1=l2normv.reshape((len(l2normv),1))
scalv=l2normv1*coefv


indtermvsp=[]
for order,ele in enumerate(scalv):
  if abs(ele)>max(abs(scalv))/50:

    indtermvsp.append(order)

lvspy=[]
for i in indtermvsp:
  lvspy.append(l2[i])

termuc={}
termvc={}

for order, ele in enumerate(luspy):
  if "/" not in str(ele):
    termuc[ele]=1

for order, ele in enumerate(lvspy):
  if "/" not in str(ele):
    termvc[ele]=1




if len(termul2)==0:
  termul2={i:[0]*simulation for i in luspy}
  termvl2={i:[0]*simulation for i in lvspy}
  for order, ele in enumerate(luspy):
    order1=indtermusp[order]
    termul2[ele][sim]=round(coefu[order1][0],5)
  for order, ele in enumerate(lvspy):
    order1=indtermvsp[order]
    termvl2[ele][sim]=round(coefv[order1][0],5)
  termul2c.update(termuc)
  termvl2c.update(termvc)
else:

  for order, ele in enumerate(luspy):
    if ele in list(termul2.keys()):
      order1=indtermusp[order]
      termul2[ele][sim]=round(coefu[order1][0],5)
    else:
      termul2[ele]=[0]*simulation
      order1=indtermusp[order]
      termul2[ele][sim]=round(coefu[order1][0],5)



  for order, ele in enumerate(lvspy):
    if ele in list(termvl2.keys()):
      order1=indtermvsp[order]
      termvl2[ele][sim]=round(coefv[order1][0],5)
    else:
      termvl2[ele]=[0]*simulation
      order1=indtermvsp[order]
      termvl2[ele][sim]=round(coefv[order1][0],5)
  for order, item in enumerate(list(termuc.keys())):
    if item in list(termul2c.keys()):
      termul2c[item]+=termuc[item]

    else:
      termul2c[item]=termuc[item]
  for order, item in enumerate(list(termvc.keys())):
    if item in list(termvl2c.keys()):
      termvl2c[item]+=termvc[item]

    else:
      termvl2c[item]=termvc[item]

In [15]:
termvl2

{u00*v00/(1.0*u00**2 + 0.252608602212288*u00 + 0.266239556147262): [-0.52235],
 1: [0.40524],
 v02 + v20: [0.39134],
 v00: [-0.19456]}

In [16]:
termul2

{1: [1.01212],
 u00: [-0.98806],
 u00*v00/(1.0*u00**2 + 0.25488950789362*u00 + 0.257304836935724): [-0.50826],
 u02 + u20: [0.29521]}

In [17]:
dataMatrixuy = np.array([rhu11[i] for i in luspy])

linear_matuy=dataMatrixuy.T.reshape((64*64*15,len(luspy)))


coefuspy=[]

for i in indtermusp:
  coefuspy.append(coefu[i])

linear_matuf1=np.round(linear_matuy,4)

coefuspy=np.reshape(coefuspy, (-1,1))

#v
dataMatrixvy = np.array([rhv11[i] for i in lvspy ])

linear_matvy=dataMatrixvy.T.reshape((64*64*15,len(lvspy)))


coefvspy=[]

for i in indtermvsp:
  coefvspy.append(coefv[i])

linear_matvf1=np.round(linear_matvy,4)

coefvspy=np.reshape(coefvspy, (-1,1))

#Sparsity

In [18]:
def TrainSTRidge(R, Ut, bestu,lam, d_tol, maxit = 35, STR_iters = 30, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.
    It runs over different values of tolerance and trains predictors on a training set, then evaluates them
    using a loss function on a holdout set.
    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001#*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = bestu
    #print(w_best)
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0
    print(err_best)
    print(np.linalg.norm(TestY - TestR.dot(w_best), 2))
    #print(np.linalg.cond(R))

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: print ("Optimal tolerance:", tol_best)

    return w_best, err_best



def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.
    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0

    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]

    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]

        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)

        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0:
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds

        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
import hyperopt
import numpy as np
import itertools
import numpy as np

from hyperopt.base import miscs_update_idxs_vals
from hyperopt.pyll.base import Apply


def recursiveFindNodes(root, node_type='switch'):
    nodes = []
    if isinstance(root, (list, tuple)):
        for node in root:
            nodes.extend(recursiveFindNodes(node, node_type))
    elif isinstance(root, dict):
        for node in root.values():
            nodes.extend(recursiveFindNodes(node, node_type))
    elif isinstance(root, (Apply)):
        if root.name == node_type:
            nodes.append(root)

        for node in root.pos_args:
            if node.name == node_type:
                nodes.append(node)
        for _, node in root.named_args:
            if node.name == node_type:
                nodes.append(node)
    return nodes


def parameters(space):
    # Analyze the domain instance to find parameters
    parameters = {}
    if isinstance(space, dict):
        space = list(space.values())
    for node in recursiveFindNodes(space, 'switch'):

        # Find the name of this parameter
        paramNode = node.pos_args[0]
        assert paramNode.name == 'hyperopt_param'
        paramName = paramNode.pos_args[0].obj

        # Find all possible choices for this parameter
        values = [literal.obj for literal in node.pos_args[1:]]
        parameters[paramName] = np.array(range(len(values)))
    return parameters


def spacesize(space):
    # Compute the number of possible combinations
    params = parameters(space)
    return np.prod([len(values) for values in params.values()])


def suggest(new_ids, domain, trials, seed):

    # Analyze the domain instance to find parameters
    params = parameters(domain.expr)

    # Compute all possible combinations
    s = [[(name, value) for value in values] for name, values in params.items()]
    values = list(itertools.product(*s))

    rval = []
    for _, new_id in enumerate(new_ids):
        # -- sample new specs, idxs, vals
        idxs = {name: np.array([new_id]) for name in params.keys()}
        vals = {name: np.array([value]) for name, value in values[new_id]}

        new_result = domain.new_result()
        new_misc = dict(tid=new_id, cmd=domain.cmd, workdir=domain.workdir)
        miscs_update_idxs_vals([new_misc], idxs, vals)
        rval.extend(trials.new_trial_docs([new_id],
                                          [None], [new_result], [new_misc]))
    return rval

In [19]:
ucoef=TrainSTRidge(linear_matuf1,uut,coefuspy,50, 10)
vcoef=TrainSTRidge(linear_matvf1,uvt,coefvspy,50, 10)

#record terms and coe with sparsity

finalindu=list(np.where(ucoef[0]!=0)[0])
fl1=[]
fcoeu=[]

for i in finalindu:
  fl1.append(luspy[i])

  fcoeu.append(ucoef[0][i].real)



finalindv=list(np.where(vcoef[0]!=0)[0])
fl2=[]
fcoev=[]

for i in finalindv:

  fl2.append(lvspy[i])

  fcoev.append(vcoef[0][i].real)

termusp1c={}
termvsp1c={}

for order, ele in enumerate(fl1):
  if "/" not in str(ele):
    termusp1c[ele]=1

for order, ele in enumerate(fl2):
  if "/" not in str(ele):
    termvsp1c[ele]=1

if len(termusp)==0:
  termusp={i:[0]*simulation for i in fl1}
  termvsp={i:[0]*simulation for i in fl2}
  for order, ele in enumerate(fl1):

    termusp[ele][sim]=round(fcoeu[order][0],5)
  for order, ele in enumerate(fl2):

    termvsp[ele][sim]=round(fcoev[order][0],5)
  termuspc.update(termusp1c)
  termvspc.update(termvsp1c)
else:

  for order, ele in enumerate(fl1):
    if ele in list(termusp.keys()):

        termusp[ele][sim]=round(fcoeu[order][0],5)
    else:
      termusp[ele]=[0]*simulation

      termusp[ele][sim]=round(fcoeu[order][0],5)



  for order, ele in enumerate(fl2):
    if ele in list(termvsp.keys()):

      termvsp[ele][sim]=round(fcoev[order][0],5)
    else:
      termvsp[ele]=[0]*simulation

      termvsp[ele][sim]=round(fcoev[order][0],5)

  for order, item in enumerate(termusp1c):
    if item in list(termuspc.keys()):
      termuspc[item]+=termusp1c[item]

    else:
      termuspc[item]=termusp1c[item]

  for order, item in enumerate(termvsp1c):
    if item in list(termvspc.keys()):
      termvspc[item]+=termvsp1c[item]

    else:
      termvspc[item]=termvsp1c[item]

1.7351113599767192
1.7311113599767192


<ipython-input-18-2cf3ee62811d>:79: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-18-2cf3ee62811d>:109: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
<ipython-input-18-2cf3ee62811d>:109: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
<ipython-input-18-2c

1.0631408164124971
1.0591408164124971


In [20]:
termusp

{1: [0.98607],
 u00: [-0.97668],
 u00*v00/(1.0*u00**2 + 0.25488950789362*u00 + 0.257304836935724): [-0.50694],
 u02 + u20: [0.29514]}

In [21]:
termvsp

{u00*v00/(1.0*u00**2 + 0.252608602212288*u00 + 0.266239556147262): [-0.52067],
 1: [0.40089],
 v02 + v20: [0.39174],
 v00: [-0.1958]}